1. Training order is such that we start with simple tones and then intersperse difficult with simple. 
2. Sample many trials, throw out repetitions and finally keep about 55/65 trials.
3. p_back = 0.3
4. End with easy trials.
5. For testing, start off with easy trials.
6. Predetermine order with 50 unique trials and repeat trials from extremes 2-3 times with the first 50 iterations. Don't repeat other trials. This will possibly help balance out the fact that we rarely see these extreme trials.
7. Checked distribution of possible tones in the regions both simple and difficult. I don't follow it but we are going to test trial tone distribution.
8. Binning coarsely with 13 distinct tones, freq discrimination is log(1003/1000)*80.



In [1]:
import math
import wave
import struct
import pdb
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.stats
import random

In [2]:
"""
Variable audio will contain a long list of samples (i.e. floating point numbers describing the
waveform).  If you were working with a very long sound you'd want to stream this to
disk instead of buffering it all in memory list this.  But most sounds will fit in 
memory.   
"""

def append_silence(audio, sample_rate, duration_milliseconds=225):
    """
    Adding silence is easy - we add zeros to the end of our array
    """
    num_samples = duration_milliseconds * (sample_rate / 1000.0)

    for x in range(int(num_samples)): 
        audio.append(0.0)
    return audio

def append_sinewave(audio, sample_rate, 
                    freq, 
                    duration_milliseconds=400, 
                    volume=1.0):
    """
    The sine wave generated here is the standard beep.  If you want something
    more aggresive you could try a square or saw tooth waveform.   Though there
    are some rather complicated issues with making high quality square and
    sawtooth waves... which we won't address here
    """ 
    num_samples = duration_milliseconds * (sample_rate / 1000.0)
    signal = []
    
    for x in range(int(num_samples)):
        signal.append(volume * math.sin(2 * math.pi * freq * (x/sample_rate)))
    signal = envelopeKCW(signal, 20, sample_rate) 
    for x_signal in range(len(signal)):
        audio.append(signal[x_signal])
    return audio

def envelopeKCW(signal,rampDur,sample_rate):
    """
    This function tries to remove the transients in the signal by enveloping the first and last period.
    Ramp duration is defined by rampDur in **ms** envelope(signal,rampDuration,samplerate)
    """
    samples = int(np.round((rampDur/1000)*sample_rate))
    x = np.arange(-np.pi,0,np.pi/samples)
    y = np.arange(0,np.pi,np.pi/samples)

    # Prepare the envelope functions
    envelope_function = np.cos(x)/2+0.5;
    
    # Fade in
    for i in range(samples):
        signal[i] = signal[i] * envelope_function[i];

    # Fade out
    for i in range(samples-1):
        current_position = len(signal) - i - 1;
        signal[current_position] = signal[current_position] * envelope_function[i+1];
    return signal


def save_wav(audio, sample_rate, file_name):
    # Open up a wav file
    wav_file=wave.open(file_name,"w")

    # wav params
    nchannels = 1
    sampwidth = 2

    # 44100 is the industry standard sample rate - CD quality.  If you need to
    # save on file size you can adjust it downwards. The standard for low quality
    # is 8000 or 8kHz.
    nframes = len(audio)
    comptype = "NONE"
    compname = "not compressed"
    wav_file.setparams((nchannels, sampwidth, sample_rate, nframes, comptype, compname))

    # WAV files here are using short, 16 bit, signed integers for the 
    # sample size.  So we multiply the floating point data we have by 32767, the
    # maximum value for a short integer.  NOTE: It is theortically possible to
    # use the floating point -1.0 to 1.0 data directly in a WAV file but not
    # obvious how to do that using the wave module in python.
    for sample in audio:
        wav_file.writeframes(struct.pack('h', int( sample * 32767.0 )))
        
    wav_file.close()
    return

def generate_tone_cloud(freq_list):
    audio = []
    sample_rate = 44100.0   
    
    freq_0 = freq_list[0]; freq_1 = freq_list[1]; freq_2 = freq_list[2]
    audio = append_sinewave(audio = audio, sample_rate = sample_rate, freq = freq_0)
    audio = append_silence(audio = audio, sample_rate = sample_rate)
    audio = append_sinewave(audio = audio, sample_rate = sample_rate, freq = freq_1)
    audio = append_silence(audio = audio, sample_rate = sample_rate)
    audio = append_sinewave(audio = audio, sample_rate = sample_rate, freq = freq_2)
    audio = append_silence(audio = audio, sample_rate = sample_rate)
    return audio, sample_rate

In [3]:
def train_task(n_trials = 10, n_tones = 3, p_low = 0.5, p_back = 0.3):
    freq_seq = np.arange(100,2000,1) #array of possible true tones
    log_freq_seq_array = np.arange(np.log10(freq_seq[0]), np.log10(freq_seq[-1]), np.log10(1003/1000)*80)
    log_freq_seq_mid = np.median(log_freq_seq_array)
    log_freq_low = [log_freq_seq_mid - 0.2,0.1]  #low freq condition is gaussian 
    log_freq_high = [log_freq_seq_mid + 0.2,0.1] #high freq condition is gaussian
    trial_tones = []
    dist_chosen = []
    kind_of_tones = []

    sample_x = np.linspace(log_freq_seq_array[0], log_freq_seq_array[-1], 101);

    lowDistWeights = scipy.stats.norm.pdf(loc = log_freq_low[0], scale = log_freq_low[1], x = sample_x);
    lowDistWeights = lowDistWeights/sum(lowDistWeights);
    highDistWeights = scipy.stats.norm.pdf(loc = log_freq_high[0], scale = log_freq_high[1], x = sample_x);
    highDistWeights = highDistWeights/sum(highDistWeights);

    bgDistWeights = scipy.stats.uniform.pdf(loc = freq_seq[0], scale = freq_seq[-1], x = 10**sample_x);
    bgDistWeights = bgDistWeights/sum(bgDistWeights);

    plt.figure()
    plt.plot(sample_x,lowDistWeights) 
    plt.plot(sample_x,highDistWeights)
    plt.plot(sample_x,bgDistWeights)
    plt.show()
    
    for trial in range(n_trials):
        signal_rand = np.random.random()
        low_dist = signal_rand < p_low #choosing true tone from either low or high condition
        tones = []
        tone_kind = []
        for n_tone in range(n_tones):
            signal_back = np.random.random()
            background = signal_back < p_back #choosing background or true tone
            if background:
                #background freq is chosen from the extremes of a uniform distribution
                if np.random.random() < 0.5:
                    nearest_log_tone = np.random.choice(log_freq_seq_array[0:3])
                else:
                    nearest_log_tone = np.random.choice(log_freq_seq_array[-4:-1])
                tone_kind.append(0)    
            else: 
                if low_dist:
                    tone = min(max(np.random.randn()*log_freq_low[1] + log_freq_low[0],\
                                   log_freq_low[0]-log_freq_low[1]),log_freq_low[0]+log_freq_low[1])
                    tone_kind.append(1)
                else:
                    tone = min(max(np.random.randn()*log_freq_high[1] + log_freq_high[0],\
                                   log_freq_high[0]-log_freq_high[1]),log_freq_high[0]+log_freq_high[1])
                    tone_kind.append(2)
                nearest_log_tone = log_freq_seq_array[np.argmin(np.abs(log_freq_seq_array - tone))]    
            nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**nearest_log_tone))]
            tones.append(nearest_tone)
        trial_tones.append(tones)
        dist_chosen.append(low_dist)
        kind_of_tones.append(tone_kind)
    return trial_tones, dist_chosen, kind_of_tones, log_freq_low, log_freq_high


In [4]:
"""
Throwing away repetitions in training and testing trials
"""

def unique(trial_list, dist_list, tone_kind_list): 
  
    # intilize a null list 
    unique_trial_list = [] 
    unique_dist_list = []
    unique_tone_kind_list = []
      
    # traverse for all elements 
    for i_element in range(len(trial_list)): 
        # check if exists in unique_list or not 
        if trial_list[i_element] not in unique_trial_list: 
            unique_trial_list.append(trial_list[i_element]) 
            unique_dist_list.append(dist_list[i_element])
            unique_tone_kind_list.append(tone_kind_list[i_element])
    return unique_trial_list, unique_dist_list, unique_tone_kind_list 
            
"""
Finding simple and difficult trials.
Come back to this, the way the if statements are phrased might not work
"""            

def find_difficult_trials(trial_list, tone_kind_list, low_dist, high_dist):
    n_tones = len(trial_list[0])
    difficult_barrier = 0
    difficult_trials = []
    simple_trials = []
    easier_trials = []
    
    for i_element in range(len(trial_list)):
        if np.sum(np.asarray(trial_list[i_element][0]) < 10**(low_dist[0]-2*low_dist[1]) and\
                  np.asarray(trial_list[i_element][1]) < 10**(low_dist[0]-2*low_dist[1]) and \
                  np.asarray(trial_list[i_element][2]) < 10**(low_dist[0]-2*low_dist[1]) or\
                  np.asarray(trial_list[i_element][0]) > 10**(high_dist[0]+2*high_dist[1]) and\
                  np.asarray(trial_list[i_element][1]) > 10**(high_dist[0]+2*high_dist[1]) and\
                 np.asarray(trial_list[i_element][2]) > 10**(high_dist[0]+2*high_dist[1])) > difficult_barrier:
            difficult_trials.append(i_element)
        elif np.sum([np.asarray(trial_list[i_element][0]) > 10**(low_dist[0]-low_dist[1]) and \
                    np.asarray(trial_list[i_element][0]) < 10**(low_dist[0]+low_dist[1]), \
                    np.asarray(trial_list[i_element][1]) > 10**(low_dist[0]-low_dist[1]) and \
                    np.asarray(trial_list[i_element][1]) < 10**(low_dist[0]+low_dist[1]),\
                     np.asarray(trial_list[i_element][2]) > 10**(low_dist[0]-low_dist[1]) and \
                    np.asarray(trial_list[i_element][2]) < 10**(low_dist[0]+low_dist[1]),\
                    np.asarray(trial_list[i_element][0]) > 10**(high_dist[0]-high_dist[1]) and \
                    np.asarray(trial_list[i_element][0]) < 10**(high_dist[0]+high_dist[1]), \
                    np.asarray(trial_list[i_element][1]) > 10**(high_dist[0]-high_dist[1]) and \
                    np.asarray(trial_list[i_element][1]) < 10**(high_dist[0]+high_dist[1]),
                    np.asarray(trial_list[i_element][2]) > 10**(high_dist[0]-high_dist[1]) and \
                    np.asarray(trial_list[i_element][2]) < 10**(high_dist[0]+high_dist[1])]) == n_tones:  
            if tone_kind_list[i_element] in [[1,1,1],[2,2,2]]:
                simple_trials.append(i_element)
        elif sum(x > 0 for x in tone_kind_list[i_element])>1:
            easier_trials.append(i_element)
    return difficult_trials, simple_trials, easier_trials

In [ ]:
orig_training_trials, orig_dist_chosen, orig_tone_kind, low_dist, high_dist = \
train_task(n_trials = 20000, n_tones = 3)

train_length = 50; n_tones = 3
"""
Throw away repetitions
"""
unique_trial_list, unique_dist_list, unique_tone_kind_list \
= unique(orig_training_trials, orig_dist_chosen, orig_tone_kind)

"""
Create order in which to present the training trials
"""
difficult_trials, simple_trials, easier_trials = find_difficult_trials(unique_trial_list, unique_tone_kind_list, \
                                                                       low_dist, high_dist)
if len(difficult_trials) > 8:
    difficult_trials_keep = np.random.choice(difficult_trials,size=8,replace=False)  
else:
    difficult_trials_keep = difficult_trials
difficult_trial_idxs = np.random.choice(np.arange(10,train_length-5),size=8,replace=False)

training_trials = np.empty([train_length,n_tones])
dist_chosen = np.zeros((train_length), dtype=bool)
tone_kind = np.empty([train_length,n_tones])

training_trials[difficult_trial_idxs] = np.asarray(unique_trial_list)[difficult_trials_keep]
dist_chosen[difficult_trial_idxs] = np.asarray(unique_dist_list)[difficult_trials_keep]
tone_kind[difficult_trial_idxs] = np.asarray(unique_tone_kind_list)[difficult_trials_keep]

initial_simple_trials = np.random.choice(simple_trials,size=10,replace=False)
training_trials[:10,:] = np.asarray(unique_trial_list)[initial_simple_trials]
dist_chosen[:10] = np.asarray(unique_dist_list)[initial_simple_trials]
tone_kind[:10] = np.asarray(unique_tone_kind_list)[initial_simple_trials]

easier_trial_idxs = np.random.choice(np.delete(np.delete(np.arange(train_length),\
                                                         difficult_trial_idxs),np.arange(10)),size=15,\
                                     replace=False)
easier_trials_keep = np.random.choice(easier_trials,size=15,replace=False)  
training_trials[easier_trial_idxs] = np.asarray(unique_trial_list)[easier_trials_keep]
dist_chosen[easier_trial_idxs] = np.asarray(unique_dist_list)[easier_trials_keep]
tone_kind[easier_trial_idxs] = np.asarray(unique_tone_kind_list)[easier_trials_keep]

remaining_trials = np.delete(np.arange(len(unique_trial_list)),\
                             np.concatenate((difficult_trials_keep,initial_simple_trials,easier_trials_keep)))
remaining_trial_idxs = np.delete(np.arange(train_length),\
                                 np.concatenate((difficult_trial_idxs,np.arange(10),easier_trial_idxs)))
remaining_trials_keep = np.random.choice(remaining_trials,size=17,replace=False)
training_trials[remaining_trial_idxs] = np.asarray(unique_trial_list)[remaining_trials_keep]
dist_chosen[remaining_trial_idxs] = np.asarray(unique_dist_list)[remaining_trials_keep]
tone_kind[remaining_trial_idxs] = np.asarray(unique_tone_kind_list)[remaining_trials_keep]

final_simple_trials = np.random.choice(simple_trials,size=5,replace=False)
training_trials = np.concatenate((training_trials, np.asarray(unique_trial_list)[final_simple_trials]))
dist_chosen = np.concatenate((dist_chosen, np.asarray(unique_dist_list)[final_simple_trials]))
tone_kind = np.concatenate((tone_kind, np.asarray(unique_tone_kind_list)[final_simple_trials]))

In [ ]:
training_trials, dist_chosen, tone_kind, \
len(unique_trial_list), len(difficult_trials), len(simple_trials), \

In [ ]:
cnt_low = 0; cnt_high = 0
rows_csv_tr = []
for i_trial in range(len(training_trials)):
    audio, sample_rate = generate_tone_cloud(list(map(int,training_trials[i_trial])))    
    if dist_chosen[i_trial]:
        save_wav(audio, sample_rate, file_name = "soundfiles_three_tones_train_v2/train_wave_low_"\
                 +str(cnt_low)+".wav")
        rows_csv_tr.append(["train_wave_low_"+str(cnt_low)+".wav", 'l', list(map(int,training_trials[i_trial]))])
        cnt_low += 1 
    else:
        save_wav(audio, sample_rate, file_name = "soundfiles_three_tones_train_v2/train_wave_high_"\
                 +str(cnt_high)+".wav")
        rows_csv_tr.append(["train_wave_high_"+str(cnt_high)+".wav", 'h', list(map(int,training_trials[i_trial]))])
        cnt_high += 1 

In [ ]:
# field names  
fields = ['Name', 'corrAns', 'Tones','Tonekind']  
    
# name of csv file  
filename = "train_three_tones_v2_file.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)          
    # writing the fields  
    csvwriter.writerow(fields)          
    # writing the data rows  
    csvwriter.writerows(rows_csv_tr) 

In [ ]:
def test_task(n_trials = 10, n_tones = 3, p_low = 0.5, p_back = 0.3):
    freq_seq = np.arange(100,2000,1) #array of possible true tones
    log_freq_seq_array = np.arange(np.log10(freq_seq[0]), np.log10(freq_seq[-1]), np.log10(1003/1000)*80)
    log_freq_seq_mid = np.median(log_freq_seq_array)
    log_freq_low = [log_freq_seq_mid - 0.2,0.1]  #low freq condition is gaussian 
    log_freq_high = [log_freq_seq_mid + 0.2,0.1] #high freq condition is gaussian
    trial_tones = []
    dist_chosen = []
    kind_of_tones = []
    
    for trial in range(n_trials):
        signal_rand = np.random.random()
        low_dist = signal_rand < p_low #choosing true tone from either low or high condition
        tones = []
        tone_kind = []
        for n_tone in range(n_tones):
            signal_back = np.random.random()
            background = signal_back < p_back #choosing background or true tone
            if background:
                nearest_log_tone = np.random.choice(log_freq_seq_array)
                #background freq is chosen from a uniform distribution
                tone_kind.append(0)
            else: 
                if low_dist:
                    tone = min(max(np.random.randn()*log_freq_low[1] + log_freq_low[0],\
                                   log_freq_seq_array[0]),log_freq_seq_array[-1])                    
                    tone_kind.append(1)
                else:
                    tone = min(max(np.random.randn()*log_freq_high[1] + log_freq_high[0],\
                                   log_freq_seq_array[0]),log_freq_seq_array[-1])
                    tone_kind.append(2)
                nearest_log_tone = log_freq_seq_array[np.argmin(np.abs(log_freq_seq_array - tone))]
            nearest_tone = freq_seq[np.argmin(np.abs(freq_seq - 10**nearest_log_tone))]        
            tones.append(nearest_tone)
        trial_tones.append(tones)
        dist_chosen.append(low_dist)
        kind_of_tones.append(tone_kind)
        
    return trial_tones, dist_chosen, kind_of_tones

In [ ]:
orig_testing_trials, orig_test_dist, orig_test_tone_kind = test_task(n_trials = 20000, n_tones = 3)

test_length = 60; n_tones = 3

"""
Throw away repetitions
"""
unique_trial_list, unique_dist_list, unique_tone_kind_list = \
unique(orig_testing_trials, orig_test_dist, orig_test_tone_kind)

"""
Create order in which to present the testing trials
"""
difficult_trials, simple_trials, easier_trials = find_difficult_trials(unique_trial_list, unique_tone_kind_list,\
                                                                       low_dist, high_dist)
if len(difficult_trials) > 10:
    difficult_trials_keep = np.random.choice(difficult_trials,size=10,replace=False)  
else:
    difficult_trials_keep = difficult_trials
difficult_trial_idxs = np.random.choice(np.arange(10,test_length),size=10,replace=False)
testing_trials = np.empty([test_length,n_tones])
test_dist = np.zeros((test_length), dtype=bool)
test_tone_kind = np.empty([test_length,n_tones])

testing_trials[difficult_trial_idxs] = np.asarray(unique_trial_list)[difficult_trials_keep]
test_dist[difficult_trial_idxs] = np.asarray(unique_dist_list)[difficult_trials_keep]
test_tone_kind[difficult_trial_idxs] = np.asarray(unique_tone_kind_list)[difficult_trials_keep]

initial_simple_trials = np.random.choice(simple_trials,size=10,replace=False)
testing_trials[:10,:] = np.asarray(unique_trial_list)[initial_simple_trials]
test_dist[:10] = np.asarray(unique_dist_list)[initial_simple_trials]
test_tone_kind[:10] = np.asarray(unique_tone_kind_list)[initial_simple_trials]

easier_trial_idxs = np.random.choice(np.delete(np.delete(np.arange(test_length),\
                                                         difficult_trial_idxs),np.arange(10)),size=20,\
                                     replace=False)
easier_trials_keep = np.random.choice(easier_trials,size=20,replace=False)  
testing_trials[easier_trial_idxs] = np.asarray(unique_trial_list)[easier_trials_keep]
test_dist[easier_trial_idxs] = np.asarray(unique_dist_list)[easier_trials_keep]
test_tone_kind[easier_trial_idxs] = np.asarray(unique_tone_kind_list)[easier_trials_keep]

remaining_trials = np.delete(np.arange(len(unique_trial_list)),\
                             np.concatenate((difficult_trials_keep,initial_simple_trials,easier_trials_keep)))
remaining_trial_idxs = np.delete(np.arange(test_length),\
                                 np.concatenate((difficult_trial_idxs,np.arange(10),easier_trial_idxs)))
remaining_trials_keep = \
np.random.choice(remaining_trials,size=test_length-(10+len(difficult_trials_keep)+len(easier_trials_keep)),\
                 replace=False)
testing_trials[remaining_trial_idxs] = np.asarray(unique_trial_list)[remaining_trials_keep]
test_dist[remaining_trial_idxs] = np.asarray(unique_dist_list)[remaining_trials_keep]
test_tone_kind[remaining_trial_idxs] = np.asarray(unique_tone_kind_list)[remaining_trials_keep]

final_simple_trials = np.random.choice(simple_trials,size=5,replace=False)
testing_trials = np.concatenate((testing_trials, np.asarray(unique_trial_list)[final_simple_trials]))
test_dist = np.concatenate((test_dist, np.asarray(unique_dist_list)[final_simple_trials]))
test_tone_kind = np.concatenate((test_tone_kind, np.asarray(unique_tone_kind_list)[final_simple_trials]))

In [ ]:
test_tone_kind

In [ ]:
"""Creating the whole testing set beforehand with the random order. Psychopy n_trials = 1"""
s = np.arange(testing_trials.shape[0])
np.random.shuffle(s)
testing_trials_1 = testing_trials[s]; test_dist_1 = test_dist[s]; test_tone_kind_1 = test_tone_kind[s]
np.random.shuffle(s)
testing_trials_2 = testing_trials[s]; test_dist_2 = test_dist[s]; test_tone_kind_2 = test_tone_kind[s]
np.random.shuffle(s)
testing_trials_3 = testing_trials[s]; test_dist_3 = test_dist[s]; test_tone_kind_3 = test_tone_kind[s]

all_testing_trials = np.concatenate((testing_trials,testing_trials_1,testing_trials_2,testing_trials_3))
all_test_dist = np.concatenate((test_dist,test_dist_1,test_dist_2,test_dist_3))
all_test_tone_kind = np.concatenate((test_tone_kind,test_tone_kind_1,test_tone_kind_2,test_tone_kind_3))

In [ ]:
cnt_low = 0; cnt_high = 0
rows_csv_tt = []
for i_trial in range(len(all_testing_trials)):
    audio, sample_rate = generate_tone_cloud(list(map(int,all_testing_trials[i_trial])))    
    if all_test_dist[i_trial]:
        save_wav(audio, sample_rate, file_name = \
                "soundfiles_three_tones_test_v2/test_wave_low_"+str(cnt_low)+".wav")
        rows_csv_tt.append(["test_wave_low_"+str(cnt_low)+".wav",'l',list(map(int,all_testing_trials[i_trial])),\
                           list(map(int,all_test_tone_kind[i_trial]))])
        cnt_low += 1  
    else:
        save_wav(audio, sample_rate, file_name = \
                "soundfiles_three_tones_test_v2/test_wave_high_"+str(cnt_high)+".wav")
        rows_csv_tt.append(["test_wave_high_"+str(cnt_high)+".wav",'h',list(map(int,all_testing_trials[i_trial])),\
                           list(map(int,all_test_tone_kind[i_trial]))])
        cnt_high += 1 

In [ ]:
# field names  
fields = ['Name', 'corrAns', 'Tones', 'Tonekind']  
    
# name of csv file  
filename = "test_three_tones_v2_file.csv"
    
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)          
    # writing the fields  
    csvwriter.writerow(fields)          
    # writing the data rows  
    csvwriter.writerows(rows_csv_tt) 